In [3]:
##버튼 생성 코드
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
from HandTrackingModule import HandDetector  # Assuming you have HandTrackingModule in the same directory
import pymunk
import time
# Pygame 초기화
pygame.init()
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear()
# 창/화면 생성
width, height = 1300, 700
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Fruit Slicer")
# FPS용 시계 초기화
fps = 23
clock = pygame.time.Clock()
# 이미지
imgGameOver = pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit/fru.jpg").convert()
# Set the desired width and height
new_width = 1300
new_height = 800
# Resize the image
imgGameOver = pygame.transform.scale(imgGameOver, (new_width, new_height))
# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
# 웹캠
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  # 높이
# 메타데이터
# fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# 웹캠 프레임의 크기에 맞는 Pygame 창 생성
window = pygame.display.set_mode((width, height))
# Hand detector 초기화
detector = HandDetector(maxHands=1, detectionCon=0.8)
# 물리 공간 초기화
space = pymunk.Space()
space.gravity = 0.0, -1000.0
# 게임 변수 및 파라미터
timeTotal = 60
fruitList = []
timeGenerator = time.time()
timeStart = time.time()
gameOver = False
score = 0
# 색상 정의
blue = (255, 127, 0)
# 과일 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
pathListFruit = os.listdir(pathFruitFolder)
def generateFruit():
    # 랜덤한 크기 설정 (0.6에서 0.8 사이)
    randomScale = round(random.uniform(0.6, 0.8), 2)
    # 랜덤한 과일 이미지 경로 선택
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]
    # 과일이 폭탄인지 확인하고, 해당하는 소리 경로 설정
    if "bomb" in randomFruitPath:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav'
    else:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'
    # Fruit 클래스를 생성하여 과일 리스트에 추가
    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))
def restart_button():
    # 'Restart' 버튼 크기 및 위치 설정
    button_width, button_height = 200, 50
    button_x, button_y = (width - button_width) // 2, height // 2 + 100
    # 'Restart' 버튼을 창에 그림 (사각형 그리기)
    pygame.draw.rect(window, (0, 255, 0), (button_x, button_y, button_width, button_height))
    # 'Restart' 버튼이 클릭되었는지 확인
    mouse_x, mouse_y = pygame.mouse.get_pos()
    click, _, _ = pygame.mouse.get_pressed()
    if button_x < mouse_x < button_x + button_width and button_y < mouse_y < button_y + button_height:
        # 'Restart' 버튼에 마우스 커서가 올라가면 색상 변경
        pygame.draw.rect(window, (0, 200, 0), (button_x, button_y, button_width, button_height))
        if click:
            # 게임 변수 초기화
            global gameOver, score, timeGenerator, timeStart, fruitList
            gameOver = False
            score = 0
            timeGenerator = time.time()
            timeStart = time.time()
            pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
            pygame.mixer.music.play(-1)
            # 새로운 과일 리스트 생성
            fruitList = []
    # 'Restart' 버튼 텍스트 표시
    font = pygame.font.Font(None, 36)
    restart_text = font.render("Restart", True, (255, 255, 255))
    text_rect = restart_text.get_rect(center=(button_x + button_width // 2, button_y + button_height // 2))
    window.blit(restart_text, text_rect)
def game_loop():
    global gameOver, score, timeGenerator, timeStart
    # 배경 음악 로드
    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
    pygame.mixer.music.play(-1)
    while cap.isOpened():
        # 이벤트 처리 (Event handling)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
        # 게임이 종료되지 않았을 때만 실행
        if gameOver is False:
            # 카메라에서 프레임 읽기
            success, img = cap.read()
            img = cv2.flip(img, 1)  # 이미지를 수평으로 뒤집기
            h, w = img.shape[:2]
            # 이미지 컬러 공간 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # MediaPipe Pose를 사용하여 포즈 처리
            keypoints = pose.process(img)
            # 이미지 컬러 공간을 다시 BGR로 변환
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            # 포즈 랜드마크 얻기
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark
            try:
                # 검지 손가락 끝의 좌표 계산
                index_finger_tip_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_tip_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
                # 손 추적 (Hand tracking)
                hands, img = detector.findHands(img)
                # 손이 감지되었을 경우, 검지 손가락의 좌표 갱신
                if hands:
                    lmlist = hands[0]['lmList']
                    ind_x, ind_y = lmlist[8][0], lmlist[8][1]
                    index_finger_tip_x, index_finger_tip_y = ind_x, ind_y
                # 검지 손가락 끝에 동그라미 그리기
                cv2.circle(img, (index_finger_tip_x, index_finger_tip_y), 20, blue, -1)
            except AttributeError:
                # 손가락 끝 좌표가 없을 경우 계속 진행
                continue
            # OpenCV 이미지 컬러 공간 변환 (BGR to RGB)
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # 이미지 시계 방향으로 90도 회전
            imgRGB = np.rot90(imgRGB)
            # Pygame Surface 생성 및 BGR을 RGB로 변환
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            # Surface 좌우로 뒤집기
            frame = pygame.transform.flip(frame, True, False)
            # Pygame 창에 프레임 표시
            window.blit(frame, (0, 0))
            # 과일 생성 주기 확인 및 과일 생성
            if time.time() - timeGenerator > 1:
                generateFruit()
                timeGenerator = time.time()
            # 손가락 좌표 갱신
            x, y = index_finger_tip_x, index_finger_tip_y
            # 과일 목록 순회
            for i, fruit in enumerate(fruitList):
                if fruit:
                    # 과일 그리기
                    fruit.draw(window)
                    # 손가락과 과일 충돌 검사
                    checkSlice = fruit.checkSlice(x, y)
                    # 충돌 결과 처리
                    if checkSlice == 2:
                        # 게임 종료
                        gameOver = True
                        pygame.mixer.music.stop()
                    if checkSlice == 1:
                        # 과일을 잘랐을 때
                        fruitList[i] = False
                        score += 1
            # 남은 시간 계산 및 갱신
            timeLeft = int(timeTotal - (time.time() - timeStart))
            # 남은 시간이 0 이하일 경우 게임 종료
            if timeLeft <= 0:
                gameOver = True
                pygame.mixer.music.stop()
            # 폰트 설정 및 점수, 남은 시간 표시
            font = pygame.font.Font(None, 60)
            textScore = font.render(str(score), True, blue)
            textTime = font.render(str(timeLeft), True, blue)
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1000, 37))
        else:
            # 게임 종료 화면 표시
            window.blit(imgGameOver, (0, 0))
            # 폰트 설정 및 You Lose! 메시지 및 점수 표시
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (0, 0, 0))
            textYour = font.render("Your Score:", True, (0, 0, 0))
            textScore = font.render(str(score), True, (0, 0, 0))
            window.blit(textLose, (400, 143))
            window.blit(textYour, (350, 243))
            window.blit(textScore, (600, 343))
            # 리스타트 버튼 그리기
            restart_button()
        # 화면 업데이트
        pygame.display.update()
        # 프레임 속도 제한 및 공간 갱신
        clock.tick(fps)
        space.step(1 / fps)
    # Pygame 종료
    pygame.quit()
# 메인 함수에서 게임 루프 실행
if __name__ == "__main__":
    game_loop()